## Make sample with less categories



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## fastai import statements for vision not including fastbook

from fastai.vision.all import *
#from fastbook import *

## Understanding length of paths

In [ ]:
## ../input/herbarium-2020-fgvc7/nybg2020/train/metadata.json
train_path = "../input/herbarium-2020-fgvc7/nybg2020/train/images/"
test_path = "../input/herbarium-2020-fgvc7/nybg2020/test/images/"
image_path = "../input/herbarium-2020-fgvc7/nybg2020/train/images/000/00/437000.jpg"

## Extracting Json contents

In [ ]:
train_json_path = train_path+"../metadata.json"
test_json_path = test_path+"../metadata.json"

In [ ]:
with open(train_json_path, encoding="utf8", errors='ignore') as f:
     tr_metadata = json.load(f)
        
# with open(test_json_path, encoding="utf8", errors='ignore') as f:
#      te_metadata = json.load(f)

In [ ]:
## Length of each of the keys!
print([(name,len(tr_metadata[name])) for name in tr_metadata.keys()])
#[(name,len(te_metadata[name])) for name in te_metadata.keys()]

In [ ]:
num = 1030746
print("annotations",tr_metadata["annotations"][num], "type", type(tr_metadata["annotations"]))
print("images",tr_metadata["images"][num])


In [ ]:
## "Licenses" is a list but "info" is not a list. Both have not more than 1 indice at max.
print("categories",tr_metadata["categories"][0:2])# index till 32000 interesting.
print("licenses",tr_metadata["licenses"],"\n")
print("regions",tr_metadata["regions"][0], "\n")
print(tr_metadata["info"], type(tr_metadata["info"]))
#print(te_metadata["info"], type(te_metadata["info"]))


In [ ]:
## Length characteristics of the meta data
n_tr_img = len(tr_metadata["annotations"])
len(tr_metadata["annotations"])==len(tr_metadata["images"]), n_tr_img/10**6

In [ ]:
im = Image.open("../input/herbarium-2020-fgvc7/nybg2020/train/images/000/00/437000.jpg")
im.to_thumb(250,250)
                

**From EDR:**
Number of Categories, Number of images per category  
[[3, 1],  
 [3726, 2],  
 [2660, 3],  
 [3769, 4],  
 [1434, 5],  
 [1243, 6],  
 [1000, 7],  
 [1833, 8],  
 [757, 9],  
 [683, 10]]  


## dict --> Df (tot_spec, min_specs, maxspec)

In [ ]:
## generate random numbers from 0 to len(pics) needed
import random

req_spec = 10000
tot_spec = 5*req_spec#len(tr_metadata["annotations"]) 
min_specimens = 2; 
max_specimens = 3;

random.seed(42)
rng_img = random.sample(range(0,n_tr_img),tot_spec) #finalnum not included
rng_img[0:5], max(rng_img), min(rng_img), n_tr_img

In [ ]:
## never grow a DF, make a list (of lists) and then convert it to pandas (https://stackoverflow.com/a/56746204/5986651)
lst_df = [[tr_metadata["annotations"][num]["category_id"], tr_metadata["annotations"][num]["image_id"], tr_metadata["images"][num]["file_name"]] for i,num in enumerate(rng_img)]
lst_df[0:5]

In [ ]:
## Convert list to DF ("category_id" and "image_id" are from "annotations". The "filepath" is from )
df = pd.DataFrame.from_records(lst_df)
df.columns  = ["category_id", "image_id", "filepath"]
df[0:10]

In [ ]:
df.dtypes

## Make basic dataframe with train and valid

In [ ]:
## Sort by length of rows
df["len_rows"] = df.groupby("category_id")[["category_id"]].transform(len)
df = df.sort_values("len_rows")
df.reset_index(drop=True, inplace=True) # otherwise you have issues with df.index
#df = df.drop("c",axis=1)
print("No. of Single images:", len(df[df["len_rows"]==1]))
print("min specimens:",df.len_rows.min(), "max_specimens:",df.len_rows.max())
df

## Modify DataFrame based on number of specimens per category

In [ ]:
## make copy of df before filtering
df_unfiltered = df.copy()

In [ ]:
## Get unique categories matching the min_spec and max_spec
if max_specimens == None: max_specimens = df.len_rows.max()
if min_specimens == None: min_specimens = df.len_rows.min()
    
df = df[(df["len_rows"] >= min_specimens) & (df["len_rows"] < max_specimens)]
df.reset_index(drop=True, inplace=True)
ctg_unq = list(df["category_id"].unique())
ctg_unq_unfiltered = df_unfiltered["category_id"].unique()

In [ ]:
## Determine valid and test dataset

train_ind_1 = [random.sample(df[df["category_id"]==ctg].index.tolist(),1) for ctg in random.sample(ctg_unq,len(ctg_unq))]
train_ind_1 = [item for sublist in train_ind_1 for item in sublist]

## get rest of train and valid dset
avl_ind = list(set(range(len(df)))-set(train_ind_1))
valid_ind = random.sample(avl_ind,int(0.2*len(df)))
train_ind = list(set(range(len(df)))-set(valid_ind))
print("Valid ind is not in Train indices?", not set(valid_ind).issubset(set(train_ind)))
print("Total images:", len(df_unfiltered), "\nTotal filtered images:", len(df), "\nTotal Categories:", len(ctg_unq_unfiltered),
      "\nTotal selected cat:", len(train_ind_1), "\nSingle images in df:", len(df[df["len_rows"]==1]))
print("Validation count:", len(valid_ind), "Training count:", len(train_ind))
print(df["len_rows"].unique()[0:10])

In [ ]:
## check: All of valid_cat should be in train_cat
valid_set = set(df.loc[valid_ind,"category_id"])
train_set = set(df.loc[train_ind,"category_id"])
print("All Valid categ in Train categories?", valid_set.issubset(train_set))
print("\nTotal Categories:", len(ctg_unq_unfiltered), "\nTotal selected cat:", len(train_ind_1), 
      "\nTotal select valid categories: ", len(valid_set),"\nTotal select Train categories:",len(train_set))
print(len(valid_set.intersection(train_set)))


In [ ]:
## Make a new column "Is_valid"
df.loc[valid_ind,["is_valid"]] = True
df.loc[train_ind,["is_valid"]] = False
df.sample(10)

In [ ]:
df.dtypes

In [ ]:
df.to_csv("df.csv")
df_unfiltered.to_csv("df_unfiltered.csv")

In [ ]:
df

## Data Block

In [ ]:
## Writting the splitter so that it valid data has categories as in train

def splitter(df):
    train_ind = df.index[df['is_valid']==False].tolist()
    valid_ind = df.index[df['is_valid']==True].tolist()
    
    valid_cats = set([df["category_id"].iloc[i] for i in valid_ind])
    train_cats = set([df["category_id"].iloc[i] for i in train_ind])
    if not valid_cats.issubset(train_cats):
        raise Exception("something is wrong")
    return train_ind,valid_ind

train,valid = splitter(df)
len(train), len(valid)

In [ ]:
def get_x(r): return "../input/herbarium-2020-fgvc7/nybg2020/train/"+r["filepath"]
def get_y(r): return r["category_id"]
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),#documentation???
    get_x = get_x,
    get_y = get_y,
    splitter=splitter,
    item_tfms=Resize(256))
    #item_tfms=RandomResizedCrop(256, min_scale=0.08),
    #batch_tfms=aug_transforms(size=224, min_scale=0.5, mult=2, pad_mode='zeros')) # next iter mult=2

    


In [ ]:
#aug_transforms??

In [ ]:
## Create dsets and dls
dsets = dblock.datasets(df)
dls = dblock.dataloaders(df,bs=128)
x,y = dsets.train[0]
#x,y,x.shape,y.shape, len(dsets.train)
#dblock.summary(df)

In [ ]:
x,y = dsets.valid[0]
x,y,x.shape,y.shape, len(dsets.valid)

In [ ]:

x1,y1 = dls.train.one_batch()
x.shape,y.shape
# x1,y1 = first(dls.train)
# x1.shape


In [ ]:
show_image(x)

In [ ]:
img = x

In [ ]:
_,axs = plt.subplots(1,3,figsize=(12,4))
for ax,method in zip(axs.flatten(), [ResizeMethod.Squish, ResizeMethod.Pad, ResizeMethod.Crop]):
    rsz = Resize(256, method=method)
    show_image(rsz(img, split_idx=0), ctx=ax, title=method);

In [ ]:
crop = RandomResizedCrop(256, min_scale=0.5)
_,axs = plt.subplots(3,3,figsize=(9,9))
for ax in axs.flatten():
    cropped = crop(img)
    show_image(cropped, ctx=ax);

In [ ]:
timg = TensorImage(array(img)).permute(2,0,1).float()/255.
def _batch_ex(bs): return TensorImage(timg[None].expand(bs, *timg.shape).clone())

#tfms = aug_transforms(pad_mode='zeros', mult=2, min_scale=0.5)
tfms = aug_transforms(size=224, min_scale=0.5, mult=2, pad_mode='zeros')

y = _batch_ex(9)
for t in tfms: y = t(y, split_idx=0)
_,axs = plt.subplots(1,3, figsize=(12,3))
for i,ax in enumerate(axs.flatten()): show_image(y[i], ctx=ax)

In [ ]:
dls.show_batch(nrows=2,ncols=3)

In [ ]:
## NN Learner 
# f1_score_multi = F1Score(average="macro") ## convert class to functie
# learn = cnn_learner(dls,resnet18,metrics=f1_score_multi)


In [ ]:
#lr1,_ = learn.lr_find()
#print(lr1)
# lr1 = 0.005

In [ ]:
## fit one cycle with freeze
#learn.fit_one_cycle(1,lr1) # no need to call freeze

In [ ]:
## find new learning rate after unfreezing
#learn.unfreeze()
#lr2, lr_st2 = learn.lr_find()


In [ ]:
# print(lr2,lr_st2)
# lr2 = max(lr2,lr_st2)


In [ ]:
## learn the rest
# learn.fit_one_cycle(10,lr_max=slice(lr2/10,lr2))# trying lr_min/100 is also an issue

In [ ]:
## NN Learner
f1_score_multi = F1Score(average="macro") ## convert class to functie
learn = cnn_learner(dls,resnet50,metrics=f1_score_multi)

In [ ]:
#resnet18

In [ ]:

learn.lr_find()
learn.fine_tune(10, base_lr=3e-3, freeze_epochs=1)

In [ ]:
#learn.lr_find()

In [ ]:
learn.lr

In [ ]:
## Export
learn.export()

In [ ]:
# Size  of files and folders
!ls -l export.pkl
!ls -l df.csv
!du -sh 